In [2]:
# import requests

# publ_webpage = requests.get('https://scs.ifmo.ru/ru/listpublication/show_all_publications.htm') # Получаем страницу со списком публикаций

with open('publ_webpage.html') as file:
    publ_webpage = file.read()
    
def transliteration2(text): # функция для транслитерации, понадобится для подсчета публикаций авторов. Source: https://www.cyberforum.ru/python/thread2218731.html
    cyrillic = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
    latin = 'a|b|v|g|d|e|e|zh|z|i|i|k|l|m|n|o|p|r|s|t|u|f|kh|tc|ch|sh|shch||y||e|iu|ia|A|B|V|G|D|E|E|Zh|Z|I|I|K|L|M|N|O|P|R|S|T|U|F|Kh|Tc|Ch|Sh|Shch||Y||E|Iu|Ia'.split('|')    
    return text.translate({ord(k):v for k,v in zip(cyrillic,latin)})
def hascyr(s): # Проверка строки на содержание кириллицы. Нужно для проверки языка публикации
    lower = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    return lower.intersection(s.lower()) != set()

In [3]:
from bs4 import BeautifulSoup as BS

bs = BS(publ_webpage, 'html')

all_publs = bs.main.find(
    "div", {"id": "list", "class": "list"}
).findAll(
    "div", {"class": "group"}
) # Получаем список публикаций, сгруппированных по годам
years = len(all_publs) # Количество лет

In [4]:
import numpy as np

all_publs_dict: dict = {}                                                                    # Словарь вида {"year": [{"authors": ["blah", "blah"], "publ_name": "Blah Blah", "journal": "Blah Blah"}, ...]}
all_publs_list: list = []                                                                    # Список публикаций
authors_dict: dict = {}                                                                      # Словарь для подсчета количества публикаций авторов
total_num_of_publs = 0                                                                       # Вводим переменную для подсчета общего количества публикаций в списке
for i in range(years):
    publs_year = all_publs[i].findAll("li")                                                  # Получаем список публикаций за год
    num_of_publs = len(publs_year)                                                           # Получаем количество публикаций за год
    total_num_of_publs += num_of_publs                                                       # Ведем подсчет общего количества публикаций за все время
    year = all_publs[i].find("div", {"class": "label"}).get_text(strip=True).split(' ')[0]   # Получаем год
    all_publs_dict[year] = []                                                                # Создаем пустой список для заполнения публикациями за год
    for j in range(num_of_publs): 
        try:
            journal = publs_year[j].find(
                "p"
            ).get_text(
                strip=True
            ).split(
                "//"
            )[1].strip(
            ).replace(
                'Подробнее', 
                '')                                                                          # Если можно получить журнал, то получаем
        except Exception as e:
            journal = np.nan                                                                 # Если журнал получить нельзя то отрабатываем Exception
            # print("ERROR: " + str(year) + ' ' + str(j) + ' ' + publs_year[j].find("p").get_text(strip=True))
            pass
        
        authors = publs_year[j].find(
            "h4"
        ).get_text(
            strip=True
        ).split(
            ','
        )
        
        publ_name = publs_year[j].find(
            "p"
        ).get_text(
            strip=True
        ).split(
            "//"
        )[0].strip()
        
        publ_lang = 'rus' if hascyr(publ_name) else 'eng'                                    # Получаем язык публикации
        
        publ_id = len(all_publs_list)
        
        all_publs_dict[year].append(                                                         # Заполняем словарь данными о публикациях
            {
                "id": publ_id,
                "authors": [transliteration2(author) for author in authors],
                "name": publ_name,
                "journal": journal,
                "year": int(year.strip()),
                "lang": publ_lang
            }
        )
        all_publs_list.append(all_publs_dict[year][-1])
        for author in authors:                                                               # Подсчитываем количество публикаций авторов
            author = transliteration2(author).strip()
            if author in authors_dict.keys():
                authors_dict[author].append(publ_id)
            else:
                authors_dict[author] = [publ_id]

In [5]:
import pandas as pd
authors_df = pd.DataFrame(
    {
        "author": list(authors_dict.keys()), 
        "value": [
            len(list(authors_dict.values())[x]) for x in range(len(list(authors_dict.values())))
        ]
    }
)

In [6]:
authors_df = authors_df.sort_values(
    by="value", 
    ascending=False,
    ignore_index = True
)
authors_df.head(5)

,author,value
0,Bogatyrev V.A.,138
1,Zhdanov D.D.,118
2,Mezhenin A.V.,78
3,Potemin I.S.,67
4,Lokalov V.A.,63


In [7]:
all_pubs_df = pd.DataFrame(
    all_publs_list, 
    columns=list(all_publs_list[0].keys())[2:]
)
#all_pubs_df.head(3)

In [8]:
import natasha
import nltk
import collections


author_sorted_counted = {}
counter = 0
for author in authors_df.head(5)["author"]:
    eng_tokens = []
    rus_tokens = []
    author_publ_df = all_pubs_df[all_pubs_df.index.isin(authors_dict[author])]
    for index, publ in author_publ_df.iterrows():
        counter += 1
        print(counter)
        if publ["lang"] == 'eng':
            tokens = nltk.word_tokenize(publ["name"])
            tagged_tokens = nltk.pos_tag(tokens)
            for token in tagged_tokens:
                if token[1].upper() in ['NN', 'NNS', 'NNP', 'NNPS', 'FW', 'JJ', 'JJR', 'JJS']:
                    eng_tokens.append(token[0] if token[0].isupper() else token[0].lower())
        else:
            doc = natasha.Doc(publ["name"])
            segmenter = natasha.Segmenter()
            emb = natasha.NewsEmbedding() 
            ner_tagger = natasha.NewsNERTagger(emb)
            morph_tagger = natasha.NewsMorphTagger(emb)
            morph_vocab = natasha.MorphVocab() 
            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            doc.tag_ner(ner_tagger)
            for token in doc.tokens:
                if token.pos in ['NOUN', 'PROPN', 'ADJ']:
                    token.lemmatize(morph_vocab)
                    rus_tokens.append(token.lemma)
    eng_counted = collections.Counter(eng_tokens)
    eng_sorted_counted = {key: value for key, value in sorted(eng_counted.items(), key=lambda item: item[1]) if value >= 5}
    rus_counted = collections.Counter(rus_tokens)
    rus_sorted_counted = {key: value for key, value in sorted(rus_counted.items(), key=lambda item: item[1]) if value >= 5}
    author_sorted_counted[author] = {k: eng_sorted_counted.get(k, 0) + rus_sorted_counted.get(k, 0) for k in set(eng_sorted_counted) | set(rus_sorted_counted)}

In [9]:
import csv

for author in list(author_sorted_counted.keys()):
    with open('./output/' + author.replace('.', '_').replace(' ', '_') + 'csv.csv', 'w') as f:
            writer = csv.writer(f, delimiter=';', quotechar='"')
            for key in author_sorted_counted[author].keys():
                writer.writerow([author_sorted_counted[author][key], key, '', ''])